In [7]:
import pandas as pd


df = pd.read_csv("output.csv")
a0 = ["hka-aqm-a017", "hka-aqm-a014"]
a1 = ["hka-aqm-a101", "hka-aqm-a102", "hka-aqm-a103", "hka-aqm-a106", "hka-aqm-a107", "hka-aqm-a108", "hka-aqm-a111", "hka-aqm-a112"]

df = df[df["device_id"].isin(a0+a1)]
df["device_id"] = df["device_id"].str.replace("hka-aqm-", "")
df["date_time"] = pd.to_datetime(df["date_time"])

In [8]:
df.to_parquet("output.parquet", index=False)